In [0]:
CREATE OR REPLACE TABLE IDENTIFIER(:catalog || '.' || :schema || '.flat') AS
WITH flattened AS (
  SELECT
    * EXCEPT(b.path),
    concat_ws(
        '\n\n',
        transform(
          try_cast(parsed:document:elements AS ARRAY<VARIANT>),
          element -> try_cast(element:content AS STRING)
        )
      ) AS text,
    concat_ws(
        ' ',
        slice(
          split(
            concat_ws(
              '\n\n',
              transform(
                try_cast(parsed:document:elements AS ARRAY<VARIANT>),
                element -> try_cast(element:content AS STRING)
              )
            ),
            ' '
          ),
          1,
          :words_preamble
        )
      ) AS preamble,
    concat_ws(
        ' ',
        slice(
          split(
            concat_ws(
              '\n\n',
              transform(
                try_cast(parsed:document:elements AS ARRAY<VARIANT>),
                element -> try_cast(element:content AS STRING)
              )
            ),
            ' '
          ),
          1,
          :words_truncated
        )
      ) AS truncated
  FROM IDENTIFIER(:catalog || '.' || :schema || '.' || :parsed_table) p
  LEFT JOIN (
    SELECT * EXCEPT(content)
    FROM IDENTIFIER(:catalog || '.' || :schema || '.bytes')
  ) b
  ON p.path = b.path
)
SELECT * FROM flattened
WHERE text != ''
;